# Recommender System

### What this project does
### What this project doesn't do

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

Until now, I've encountered two types of data objects in the Pandas library—DataFrames and Series, and here is a [Scaler Topics article on core components of Pandas](https://www.scaler.com/topics/pandas/core-components-of-pandas/), confirming that these two are the important ones (for single-dimensional, and two-dimensional data respectively), apart from a third object type, `Panel` (for three-dimensional data).

Here's a micro-article attempting to list the organisation of files in Pandas' architecture: [Discover Pandas Library Architecture – File Hierarchy in Pandas](https://data-flair.training/blogs/pandas-library-architecture/)


In [2]:
ratings_df = pd.read_csv("./ml-latest-small/ratings.csv")
# ratings_df = pd.read_csv("./anime-ratings-matrix-factorization-v-10/anime ratings dataset/rating.csv")

In [3]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [5]:
# num_movies = pd.DataFrame(ratings_df['movieId'].unique()).count().values[0]       #this is a tad more convoluted way
# num_users = pd.DataFrame(ratings_df['userId'].unique()).count().values[0]
num_movies = ratings_df['movieId'].unique().shape[0]                                #better way
num_users = ratings_df['userId'].unique().shape[0]
print(f"There are {num_movies} movies and {num_users} users in the dataset")

There are 9724 movies and 610 users in the dataset


In [6]:
print(f"There are {ratings_df.loc[ratings_df.userId > 610].shape[0]} user IDs exceeding 610.")
print(f"There are {ratings_df.loc[ratings_df.movieId > 9724].movieId.unique().shape[0]} movie IDs exceeding 9724.")

There are 0 user IDs exceeding 610.
There are 4334 movie IDs exceeding 9724.


It seems that the user IDs are contiguous, while the move IDs are not. We might have to consider making them contiguous.

In [7]:
ratings_df.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,100836.0,3.261276e+02,1.826185e+02,1.0,1.770000e+02,3.250000e+02,4.770000e+02,6.100000e+02
movieId,100836.0,1.943530e+04,3.553099e+04,1.0,1.199000e+03,2.991000e+03,8.122000e+03,1.936090e+05
rating,100836.0,3.501557e+00,1.042529e+00,0.5,3.000000e+00,3.500000e+00,4.000000e+00,5.000000e+00
timestamp,100836.0,1.205946e+09,2.162610e+08,828124615.0,1.019124e+09,1.186087e+09,1.435994e+09,1.537799e+09


In [8]:
pd.Series.value_counts(ratings_df['rating']).to_frame().T

rating,4.0,3.0,5.0,3.5,4.5,2.0,2.5,1.0,1.5,0.5
count,26818,20047,13211,13136,8551,7551,5550,2811,1791,1370


In [9]:
# plt.hist(ratings_df['rating'], bins=[(i-1)/2 for i in range(0, 12)])
# plt.hist(ratings_df['rating'], bins=[i-0.5 for i in range(-1,12)])  #for anime ratings dataset

In [10]:
print( ratings_df.isna().sum() , "\n")
print( type(ratings_df.isna().sum()) )

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64 

<class 'pandas.core.series.Series'>


In [12]:
print( len(ratings_df.duplicated(subset=['userId', 'movieId'])) )
len(np.where( ratings_df.duplicated(subset=['userId', 'movieId']) == False )[0])

100836


100836

There don't seem to be any duplicates *shruggie

In [13]:
toy = np.zeros((3,4))
toy[0,1] = 1
toy[2,3] = 2
toy

array([[0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 2.]])

In [14]:
toy_csc = sparse.csc_matrix(toy)
print( toy_csc.data )
print( toy_csc.nnz )  # nnz => no. of non-zero (entries)
print( toy_csc.nonzero() )

[1. 2.]
2
(array([0, 2], dtype=int32), array([1, 3], dtype=int32))


In [150]:
toy_csc.power(2).todense()

matrix([[0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 4.]])

In [100]:
movie_old_new_id = np.zeros(ratings_df["movieId"].max()+1)
for new_id, old_id in enumerate(ratings_df['movieId'].unique()):
       movie_old_new_id[old_id] = new_id
       
ratings_df['movieNewId'] = movie_old_new_id[ratings_df['movieId']]
ratings_df = ratings_df.astype({'movieNewId': 'int64'})
print( ratings_df.head(), "\n" )

# now just to check that we did it correctly, a movieId should be mapped to the same movieNewId
print( ratings_df.loc[ratings_df.movieId == 156][['movieId', 'movieNewId']], "\n" ) 
print(f"There are {num_movies} movies and correspondingly {max(movie_old_new_id)+1} contiguous movie IDs now")

   userId  movieId  rating  timestamp  movieNewId
0       1        1     4.0  964982703           0
1       1        3     4.0  964981247           1
2       1        6     4.0  964982224           2
3       1       47     5.0  964983815           3
4       1       50     5.0  964982931           4 

       movieId  movieNewId
17007      156        4463
96137      156        4463
97391      156        4463 

There are 9724 movies and correspondingly 9724.0 contiguous movie IDs now


In [101]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
print(train_df)
print(test_df)
print( np.shape(train_df), np.shape(test_df) )

       userId  movieId  rating   timestamp  movieNewId
0          64     3101     3.5  1161521521        1685
1         354     1997     4.0  1200952925        2904
2         564    74946     4.0  1478451898        3471
3         608       44     0.5  1117504562         971
4          66     4149     4.0  1113190565        3237
...       ...      ...     ...         ...         ...
80663      28    39381     3.5  1234338567        2294
80664     605      314     3.0  1277176056         595
80665      89     1394     2.0  1520408298        1474
80666      30    59315     5.0  1500370444        1054
80667      64     1204     5.0  1161529352        2395

[80668 rows x 5 columns]
       userId  movieId  rating   timestamp  movieNewId
0         409     1125     3.0   967920162        2997
1          47       31     3.0  1496205717         259
2         401     4306     3.5  1514347054         744
3         473     5785     3.5  1169351797        1850
4         200      357     4.0  1229886

Since we've chosen to make the movie IDs contiguous by mapping them to new ones, splitting the DataFrame makes creating smaller-dimensional matrix representations of the training or test sets difficult, because, say, the indices chosen in the training set can still lie out of bounds of its smaller matrix's size (similar logic holds for the test set).

Instead, let's just work with the full ratings matrix, with the test entries zeroed out in the training matrix (and vice versa), which wouldn't be a bother to the memory anyway because we're using a sparse representation!

In [102]:
train = sparse.csc_matrix( (train_df['rating'].values, (train_df['userId'].values, train_df['movieNewId'].values)),\
                                 shape=(num_users+1, num_movies+1) )
test = sparse.csc_matrix(  (test_df['rating'].values, (test_df['userId'].values, test_df['movieNewId'].values)),\
                                 shape=(num_users+1, num_movies+1))

print(f"There are {train.nnz} & {test.nnz} non-zero elements in the train and test sparse matrices respectively.\n", test.shape[0],train.shape[1] )
# train.to_dense()
# print( train )

There are 80668 & 20168 non-zero elements in the train and test sparse matrices respectively.
 611 9725


Lemme check once that the test and train matrix elements are indeed mutually exclusive (zero, non-zero in their matrices respectively).

The cell below seems to show all's good

In [103]:
# np.shape( test.nonzero() )
# print( test.nonzero() )
for i in zip(test.nonzero()[0], test.nonzero()[1]):
       if( train[i[0],i[1]] != 0 ):
              print("Hah!")
for i in zip(train.nonzero()[0], train.nonzero()[1]):
       if( test[i[0],i[1]] != 0 ):
              print("Hah!")

In [112]:
print( np.sort(train_df['userId'].values) )
print( mf.p_nnz_idx, "\n" )

print( np.sort(train_df['movieNewId'].values) )
print( np.sort(mf.q_nnz_idx ) )
print( np.sort(train_df['userId'].values) == mf.p_nnz_idx )
print( np.sort(train_df['movieNewId'].values) == np.sort(mf.q_nnz_idx), "\n" )

print( np.where( np.sort(train_df['userId'].values) != train.nonzero()[0]) )
print( np.where( np.sort(train_df['movieNewId'].values) != np.sort( train.nonzero()[1]) ) )
print( np.where( np.sort(train_df['movieNewId'].values) != np.sort(mf.q_nnz_idx)) )


[  1   1   1 ... 610 610 610]
[  1   1   1 ... 610 610 610] 

[   0    0    0 ... 9720 9721 9723]
[   0    0    0 ... 9720 9721 9723]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True] 

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)


Hmmm, so the row indices seem to be all good, ~~but the column indices are playing foul.~~ as do the column indices.

Wasted quite the time pursuing this!

We have used the list `train_df['movieNewId'].values` as the column indices needed to generate R, but upon querying back these column indices, they're coming out wrong!
***

## Matrix Factorisation Class
Below, I furnish a Matrix Factorisation Python class that implements:
- gradient descent
- (pending) sgd
- (pending) ALS
to learn the embeddings.

My current implementation of gradient descent considers the loss function for explicit ratings, i.e. solely over observed values.
Still need to implement the Koren et al. approach capturing implicit ratings from the entire matrix.

The regularised loss I've used for explicit ratings: $L(R, P, Q) = \underset{(i,j):r_{ui}≠Nan}{\sum} (r_{ui} - p_u^T\cdot q_i)^2 + \lambda(\sum_u{||p_u||^2} + \sum_i{||q_i||^2})$
It's a squared error (SE) loss function with regularisation terms. I wonder how using the Mean SE or RMSE would change the behaviour. Conceptually, it shouldn't..

The matrix product used to calculate the prediction for the error term in the gradient can be further optimised, as done in the other notebook, but for now, I'm just gonna roll with a matrix product.

In [207]:
class MatrixFactorisation():
       def __init__(self, R: sparse.csc_matrix, k: int):
              self.R = R           # R is the "ratings" matrix to be factorised
              self.k = k           # hyperparameter k is the number of latent factors desired in the factorised embeddings
              self.m, self.n = R.shape
              self.P = np.random.rand(self.m, k)        # initialising the first factor embedding (of "users")
              self.Q = np.random.rand(self.n, k)        # initialising the second factor embedding (of "items")
              self.p_nnz_idx, self.q_nnz_idx = R.nonzero()        # row and col indices in R matrix respectively, of non-zero elements

       def __predict_observed(self):
              # print(np.shape(P[p_nnz_idx]), np.shape(Q[q_nnz_idx]))
              R_predicted = np.sum( np.multiply(self.P[self.p_nnz_idx], self.Q[self.q_nnz_idx]), axis=1 )       #calculating the products only for the nonzero "observed" rating entries
              R_predicted = sparse.csc_matrix( (R_predicted, (self.p_nnz_idx, self.q_nnz_idx)), shape=(self.m,self.n))

              # print( np.shape(R_predicted))
              # print(R_predicted)
              
              return R_predicted

       # def __predict_observed(self):
       #        R_predicted = self.P @ self.Q.T
       #        R_predicted = sparse.csc_matrix(R_predicted)
       #        return R_predicted

       def gradients(self, R, _lambda):
              """
              returns gradients w.r.t the elements of the embedding vectors
              """
              # R_predicted = self.__predict_observed(self.R, self.P, self.Q)
              R_predicted = self.__predict_observed()
              error = R - R_predicted

              # print(type(error), type(R), type(R_predicted))
              # print( np.shape(error * self.Q), np.shape(_lambda*self.P) )
              # print( type(error), type(self.Q), type(error * self.Q) )
              # print(np.shape(error), type(error))
              # print(error)
              norm_factor = -2 / self.R.shape[0]
              grad_P = norm_factor * (error*self.Q) + 2*(_lambda*self.P)
              grad_Q = norm_factor * (error.T*self.P) + 2*(_lambda*self.Q)
              return grad_P, grad_Q, error

       def gradient_descent(self, iterations=2000, learning_rate = 0.1, _lambda=0):
              grad_P, grad_Q, error = self.gradients(self.R, _lambda)
              v_P = grad_P
              v_Q = grad_Q
              beta = 0.8    # momentum
              print("train mse:", error.power(2).sum()/self.R.shape[0] )
              for i in range(iterations):
                     grad_P, grad_Q, error = self.gradients(self.R, _lambda)
                     v_P = beta*v_P + (1-beta)*grad_P
                     v_Q = beta*v_Q + (1-beta)*grad_Q
                     self.P = self.P - learning_rate*v_P
                     self.Q = self.Q - learning_rate*v_Q
                     # print(np.shape(error))
                     # print("train mse:", error.power(2).sum()/self.R.shape[0] )
                     if(not (i+1)%50):
                            print("\niteration", i+1, ":")
                            print("train mse:", error.power(2).sum()/self.R.shape[0] )
              return self.P, self.Q

In [208]:
mf = MatrixFactorisation(train, 10)
print( np.shape(mf.P), np.shape(mf.Q) )
# print( mf.P )

(611, 10) (9725, 10)


In [214]:
P, Q = mf.gradient_descent(iterations=2000, learning_rate=0.1, _lambda=0)
print(np.shape(P))

train mse: 47.639789412277345

iteration 50 :
train mse: 46.69468618603528

iteration 100 :
train mse: 45.80448384130397

iteration 150 :
train mse: 44.98197347231309

iteration 200 :
train mse: 44.220294237190714

iteration 250 :
train mse: 43.51333647320343

iteration 300 :
train mse: 42.85568461577408

iteration 350 :
train mse: 42.24255160164141

iteration 400 :
train mse: 41.669710809924794

iteration 450 :
train mse: 41.13342965630901

iteration 500 :
train mse: 40.63040744681738

iteration 550 :
train mse: 40.15771896564603

iteration 600 :
train mse: 39.71276444337787

iteration 650 :
train mse: 39.29322596721012

iteration 700 :
train mse: 38.89703000636026

iteration 750 :
train mse: 38.522315493462486

iteration 800 :
train mse: 38.16740679016967

iteration 850 :
train mse: 37.83079083930364

iteration 900 :
train mse: 37.51109783791531

iteration 950 :
train mse: 37.20708483176317

iteration 1000 :
train mse: 36.91762171367141

iteration 1050 :
train mse: 36.64167919295731


In [215]:
# help = np.zeros((mf.P.shape[0], mf.Q.T.shape[1]))
# print(np.shape(help))

# for i in range(mf.P.shape[0]):
#        for j in range(mf.Q.T.shape[1]):
#               for k in range(mf.k):
#                      help[i][j] += mf.P[i][k] * mf.Q.T[k][j]

# print(help)

In [216]:
# help2 = mf.P @ mf.Q.T
print(mf.P,"\n\n\n\n", mf.Q)

[[0.41100543 0.75584689 0.08117057 ... 0.92413234 0.5204829  0.15446489]
 [1.00943587 0.16706719 1.47057236 ... 0.8999667  0.42411668 1.00865632]
 [0.96551131 0.09642666 0.50392217 ... 0.96584484 0.28503007 0.95188904]
 ...
 [1.36643691 0.21832797 1.15372701 ... 0.86486232 0.24416721 0.3644143 ]
 [0.8044081  0.01076732 0.37791874 ... 0.57012416 0.03161045 1.55030358]
 [0.56078029 1.27169228 0.98690866 ... 0.53588961 0.45196859 0.77140536]] 



 [[0.16367328 0.70449521 0.41476788 ... 0.40783311 0.75031928 0.62704698]
 [1.11850844 0.72464111 0.33233906 ... 0.59692173 0.84342571 0.47134425]
 [0.8282649  0.89514434 0.47506307 ... 0.64428319 0.33005133 0.6373232 ]
 ...
 [0.17427384 0.59094777 0.96109792 ... 0.15780478 0.07836124 0.74024984]
 [0.27044104 0.72316968 0.60561996 ... 0.35376109 0.29613155 0.65295007]
 [0.16502885 0.04639174 0.87177167 ... 0.87485442 0.46799892 0.60803272]]


In [217]:
train.todense()

matrix([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [4. , 4. , 4. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [5. , 0. , 5. , ..., 0. , 3.5, 0. ]])

In [218]:
predictions = mf.P.dot(mf.Q.T)
mask = np.zeros_like(train.toarray())
print(type(predictions), type(train.toarray()), train.toarray().nonzero())
mask[train.toarray().nonzero()] = 1

# Mask out unknown ratings as 0 for ease of comparison.
print(np.round(predictions * mask, 2))


<class 'numpy.ndarray'> <class 'numpy.ndarray'> (array([  1,   1,   1, ..., 610, 610, 610]), array([   0,    1,    2, ..., 9720, 9721, 9723]))
[[0.   0.   0.   ... 0.   0.   0.  ]
 [4.55 3.55 4.76 ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [3.48 0.   0.   ... 0.   0.   0.  ]
 [4.26 0.   4.41 ... 0.   3.52 0.  ]]
